In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Get the Data
** Read the 'KNN_Project_Data csv file into a dataframe **

In [93]:
df = pd.DataFrame(
    [['Conventional', 'One-to-four family dwelling (other than manufactured housing)', 'Home purchase', 'Owner-occupied as a principal dwelling', 396, 'Loan originated', 'Female', 152]], columns=['loan_type', 'property_type', 'purpose', 'occupancy', 'amount', 'status', 'sex', 'income'])


**Check the head of the dataframe.**

In [94]:
df.head()

,loan_type,property_type,purpose,occupancy,amount,status,sex,income
0,Conventional,One-to-four family dwelling (other than manufa...,Home purchase,Owner-occupied as a principal dwelling,396,Loan originated,Female,152


In [95]:
df.columns

Index(['loan_type', 'property_type', 'purpose', 'occupancy', 'amount',
       'status', 'sex', 'income'],
      dtype='object')

# EDA

In [96]:
# Remove null values
df = df.dropna()

### Status

In [97]:
# Keep only records that either granted or rejected loans
states = ['originated', 'approved', 'denied']
df = df[df.apply(lambda x: any(s in x.status for s in states), axis=1)]

In [98]:
df.status.value_counts()

Loan originated    1
Name: status, dtype: int64

In [99]:
# Merged approved and withdrawn in row status
def f(row):
    return 1 if ("approved" in row.status or "originated" in row.status) else 0
df['status'] = df.apply(f, axis=1)


In [100]:
df['status'].unique()

array([1])

In [101]:
# Making sure the counts added up
df.status.value_counts()

1    1
Name: status, dtype: int64

# Encode Categorical Features

In [102]:
df.dtypes

loan_type        object
property_type    object
purpose          object
occupancy        object
amount            int64
status            int64
sex              object
income            int64
dtype: object

In [103]:
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(df)
categorical_columns
df = pd.get_dummies(df, columns=categorical_columns)
df.head()


,amount,status,income,loan_type_Conventional,property_type_One-to-four family dwelling (other than manufactured housing),purpose_Home purchase,occupancy_Owner-occupied as a principal dwelling,sex_Female
0,396,1,152,1,1,1,1,1


In [104]:
categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(df)

# Standardize the Variables

Time to standardize the variables.

** Import StandardScaler from Scikit learn.**

In [105]:
from sklearn.preprocessing import StandardScaler

** Create a StandardScaler() object called scaler.**

In [106]:
scaler = StandardScaler()

** Fit scaler to the features.**

In [107]:
scaler.fit(df.drop('status', axis=1))

StandardScaler()

**Use the .transform() method to transform the features to a scaled version.**

In [108]:
scaled_features = scaler.transform(df.drop('status', axis=1))


**Convert the scaled features to a dataframe and check the head of this dataframe to make sure the scaling worked.**

In [109]:
df_features = pd.DataFrame(scaled_features, columns=[col for col in df.columns if col != 'status'])
df_features.head()


,amount,income,loan_type_Conventional,property_type_One-to-four family dwelling (other than manufactured housing),purpose_Home purchase,occupancy_Owner-occupied as a principal dwelling,sex_Female
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
X = df.drop(['status'], axis=1)
Y = df['status']

from sklearn.preprocessing import OneHotEncoder

# Create the encoder.
encoder = OneHotEncoder(handle_unknown="ignore")
encoder.fit(X) 

X = encoder.transform(X)

In [111]:
X.shape

(1, 7)

In [112]:
Y.shape

(1,)

In [113]:
import pickle
model = pickle.load(open('model', 'rb'))
result = model.predict(X)
result

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 7 while Y.shape[1] == 2228